In [ ]:
CREATE OR REPLACE NETWORK RULE EN_NETWORK_RULE
MODE = EGRESS
TYPE = HOST_PORT
VALUE_LIST = ('US.ENGAGINGNETWORKS.APP');

In [ ]:
CREATE OR REPLACE SECRET EN_API_TOKEN_DB
TYPE = GENERIC_STRING

SECRET_STRING='a8fda2f8-aff5-44e7-be10-6234309f3ce0' --Snowflake_EN_API user
--SECRET_STRING='63f77a9d-266b-4063-ac29-d548fa9bfe57';   --Daina
--SECRET_STRING ='8a911743-eff6-40fd-b4ba-848a17f4ca9e';    --Jonathan

In [ ]:
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION EN_INTEGRATION
ALLOWED_NETWORK_RULES = (EN_NETWORK_RULE)
ALLOWED_AUTHENTICATION_SECRETS = (EN_API_TOKEN_DB)
ENABLED=TRUE;

In [ ]:
CREATE OR REPLACE ROLE EN_DEVELOPER;
GRANT USAGE ON INTEGRATION EN_INTEGRATION TO ROLE EN_DEVELOPER;
GRANT READ ON SECRET EN_API_TOKEN_DB TO ROLE EN_DEVELOPER;

In [ ]:
CREATE OR REPLACE FUNCTION PROCESS_EN_PAGE(PAGE_ID STRING, EMAIL_ADDRESS STRING)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = 3.11
HANDLER = 'process_en_page'
EXTERNAL_ACCESS_INTEGRATIONS = (EN_INTEGRATION)
PACKAGES = ('requests')
SECRETS = ('cred' = EN_API_TOKEN_DB)
AS
$$
import requests
import _snowflake

def process_en_page(page_id, email_address):

    # URL for the authentication endpoint
    auth_url = "https://us.engagingnetworks.app/ens/service/authenticate"
    
    # Get the API token from Snowflake Secrets
    api_token = _snowflake.get_generic_secret_string('cred')  

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json"
    }

    # Authenticate and get the session token
    response = requests.post(auth_url, data=api_token, headers=headers)

    try:
        auth_data = response.json()  # Convert to JSON
    except ValueError:  # In case JSON decoding fails
        return f"JSON decoding failed: {response.text}"  # Return raw response text if not JSON

    # Validate if 'ens-auth-token' is present
    if 'ens-auth-token' not in auth_data:
        return f"'ens-auth-token' not found in response: {auth_data}"
    
    # Extract session token from the response
    session_token = auth_data['ens-auth-token']
    
    # Base URL for the Engaging Networks API to process supporters to a page
    process_url = f"https://us.engagingnetworks.app/ens/service/page/{page_id}/process"
    
    # Define the headers, including the session token for authorization
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "ens-auth-token": session_token
    }
    
    # Create the supporter object containing the email address
    supporter = {
        "suppressAutoResponder": True,
        "standardFieldNames": False,
        "supporter": {
            "Email Address": email_address
        }
    }
    
    try:
        # Make the POST request to process the supporter on the specified page
        response = requests.post(process_url, json=supporter, headers=headers)
        response.raise_for_status()  # Ensure that the request was successful
        
        # Return the full JSON response content
        return response.json()  # Convert the response to JSON and return it
    
    except requests.exceptions.HTTPError as err:
        return f"Failed to process email {email_address} for page {page_id}: {err}, Response: {response.text}"

$$;


In [ ]:
import snowflake.snowpark as snowpark
import pandas as pd

In [ ]:
# The Snowpark session is automatically managed within Snowflake
session = snowpark.Session.builder.getOrCreate()

In [ ]:
# Load data from the Snowflake stage into a DataFrame
df = session.read.options({"field_delimiter": ",", "skip_header": 1}).csv("@TEST/P2P_EN_page_interactions.csv")
df_pd = df.to_pandas()
df_pd.columns = ['EMAIL_ADDRESS', 'EN_PAGE_ID']
df_pd.dtypes

In [ ]:
for index, row in df_pd.iterrows():
    email = row['EMAIL_ADDRESS']
    page_id = row['EN_PAGE_ID']
    
    # Run the UDF through an SQL query
    query = f"""
        SELECT PROCESS_EN_PAGE('{page_id}', '{email}') AS result
    """
    
    # Execute the query
    result = session.sql(query).collect()
    
    # Display the result for each email/page processed
    print(result)